In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train_user = pd.read_csv('data/raw/tianchi_mobile_recommend_train_user.csv')
df_train_user

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21
5,93784494,337869048,1,NaN,3979,2014-12-03 20
6,94832743,105749725,1,NaN,9559,2014-12-13 20
7,95290487,76866650,1,NaN,10875,2014-11-27 16
8,96610296,161166643,1,NaN,3064,2014-12-11 23
9,100684618,21751142,3,NaN,2158,2014-12-05 23


In [3]:
# # 过滤掉 12.12 的记录
# df_train_user = df_train_user[df_train_user.time.map(lambda x: not x.startswith('2014-12-12'))]
# df_train_user.to_csv('data/raw/no1212tianchi_mobile_recommend_train_user.csv', index=False)

分割点为 2014-12-18 00  
训练集train_set，11-18～12-17，其相应的类别标签（tag：1,购买; 0,未购买）来自于12-18日的购买情况，各个特征以此时间段计算出  
测试集test_set，11-18～12-18，没有类别标签，特征由此段时间计算出  
验证集validate_set，12-18，这一天的购买了的数据，字段只要包含(user_id,item_id) 候选集predict_set如何产生  

In [4]:
import datetime

def ge_validate_set(df, separate_day='2014-12-18 00', to_file='data/raw_validate_set.csv'):
    # TODO: 使用datetime格式替代str格式来比较
    # separate_day = datetime.datetime(2014,12,18,0)
    # parseDate = lambda s: datetime.datetime.strptime(s, "%Y-%m-%d %H")
    validate_set = df[ df.time >= '2014-12-18 00']
    validate_set = validate_set[ df.behavior_type > 3]
    # TODO: 去重
    if to_file:
        validate_set.to_csv(to_file, index=False)
    return validate_set


def ge_result(df=None, to_file='data/result.csv'):
    # 结果只保留 user_id, item_id
    result = df or pd.read_csv('data/raw_validate_set.csv')
    #result = result.drop(['behavior_type','user_geohash','item_category','time'], axis=1)
    result = result[['user_id', 'item_id']]
    # 过滤出在P商品子集上的记录
    P = pd.read_csv('data/raw/tianchi_mobile_recommend_train_item.csv')
    P_item_set = set(P.item_id.values)
    result = result[result.item_id.map(lambda x: x in P_item_set)]
    # 去重
    result = result.groupby(['user_id', 'item_id'])
    result = pd.DataFrame(result.groups.keys(), columns=['user_id', 'item_id'])
    if to_file:
        result.to_csv(to_file, index=False)
    return result


# train_set：
# 1. 过滤出time<2014-12-18 00的所有记录
# 2. 对过滤出的记录添加buy column(根据12-18号的记录对其进行标记)
# 3. 要把time字段替换为date_delta -> 放到特征工程中再做吧
def ge_train_set(df, result, separate_day='2014-12-18 00', to_file='data/1212raw_train_set.csv'):
    # TODO: 使用datetime格式替代str格式来比较
    # separate_day = datetime.datetime(2014,12,18,0)
    # parseDate = lambda s: datetime.datetime.strptime(s, "%Y-%m-%d %H")
    result_set = set((i[0],i[1]) for i in result.values)
    train_set = df[ df.time < '2014-12-18 00']
    
    if to_file:
        train_set.to_csv(to_file, index=False)
    return train_set


def ge_test_set(df, to_file='data/test_set/tianchi_mobile_recommend_test_user.csv'):
    # 商品子集 P
    P = pd.read_csv('data/raw/tianchi_mobile_recommend_train_item.csv')
    P_item_set = set(P.item_id.values)
    # 过滤出 item_id 属于 P(商品子集) 的记录
    test_set = df[df.item_id.map(lambda x: x in P_item_set)]
    if to_file:
        test_set.to_csv(to_file, index=False)
    return test_set

###验证集validate_set

12-18，这一天的购买了的数据，字段只要包含(user_id,item_id) 候选集predict_set如何产生

In [5]:
validate_set = ge_validate_set(df_train_user)
validate_set

/home/kuroro/.virtualenvs/ml/local/lib/python2.7/site-packages/pandas/core/frame.py:1815: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,user_id,item_id,behavior_type,user_geohash,item_category,time
1421,132252054,319438796,4,NaN,2153,2014-12-18 16
12336,2242451,59801863,4,NaN,3942,2014-12-18 10
15169,140781295,265069069,4,NaN,11279,2014-12-18 12
16053,35354529,8080117,4,NaN,5232,2014-12-18 17
20237,4361577,200128346,4,95b03su,7405,2014-12-18 19
22027,32198657,343998250,4,NaN,9765,2014-12-18 19
28892,98607707,399424279,4,NaN,3628,2014-12-18 12
36075,102057111,60329480,4,NaN,2993,2014-12-18 11
38454,137204928,378584526,4,NaN,1083,2014-12-18 10
38824,110883373,386103471,4,99nfp0i,8010,2014-12-18 23


In [6]:
#result = ge_result(validate_set)
result = ge_result()
result

,user_id,item_id
0,109179391,227275517
1,99418963,366911878
2,80519771,81363321
3,23228065,197445650
4,33173119,9466715
5,78574430,350898954
6,118282748,90395886
7,29285910,171315964
8,62443822,368529623
9,53517495,313891623


###训练集train_set

11-18～12-17，其相应的类别标签（tag：1,购买; 0,未购买）来自于12-18日的购买情况，各个特征以此时间段计算出

In [7]:
train_set = ge_train_set(df_train_user, result)
train_set

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21
5,93784494,337869048,1,NaN,3979,2014-12-03 20
6,94832743,105749725,1,NaN,9559,2014-12-13 20
7,95290487,76866650,1,NaN,10875,2014-11-27 16
8,96610296,161166643,1,NaN,3064,2014-12-11 23
9,100684618,21751142,3,NaN,2158,2014-12-05 23
10,100509623,266020206,3,tfvomgk,4923,2014-12-08 17


###测试集test_set

In [8]:
test_set = ge_test_set(df_train_user)
test_set

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
8,96610296,161166643,1,NaN,3064,2014-12-11 23
9,100684618,21751142,3,NaN,2158,2014-12-05 23
17,110418606,144902506,1,NaN,11639,2014-12-09 16
21,103891828,149380817,1,NaN,7876,2014-12-08 21
24,103802946,142178953,1,NaN,12536,2014-11-23 21
33,103802946,194298205,1,NaN,11406,2014-12-18 21
38,101781721,5166487,1,9rgt17v,9829,2014-12-18 14
39,103891828,145981615,1,NaN,7876,2014-12-11 08
